# A tutorial on kernel compatibility in Python: kernel-target and task-model alignments




I stumbled in this article:

> Canatar, Abdulkadir, Blake Bordelon, and Cengiz Pehlevan. "Spectral bias and task-model alignment explain generalization in kernel regression and infinitely wide neural networks." Nature communications 12.1 (2021): 1-12.

The purpose of this blog post is illustrating this wonderful approach and writing a little code to give a warm start. 

We setup the Python environment:

In [306]:
# linear algebra library
import numpy as np
# advanced scientific programming library
import scipy
from scipy.linalg import eigh
from scipy.spatial.distance import cdist
# machine learning utilities, e.g. loading some datasets
from sklearn.datasets import fetch_california_housing
# for reproducibility purposes, fix a random seed at the beginning
np.random.seed(1234)

## Problem setup

Consider a set of empirical data $x^{(1)}, ..., x^{(M)} \in \mathcal{X} \subseteq \mathbb{R}^P$, drawn i.i.d. from an unknown probability density function $p$, and its corresponding labels $y^{(i)} \in \mathbb{R}, y^{(i)} = f(x^{(i)}) + \epsilon^{(i)}$ generated from a target function $f$ plus an addictive white noise (drawn from a normal distribution of zero mean and variance $\sigma^2$).

For this tutorial we consider the California housing dataset, composed of $M = 20640$ items having $P = 8$ features.

In [107]:
dataset = fetch_california_housing()
X_dataset = dataset['data']    # np.array of size (20640, 8)
y_dataset = dataset['target']  # np.array of size (20640,)

There are too many samples, we can reduce it

In [408]:
M = 50
X = X_dataset[:M]
y = y_dataset[:M]

## Kernel models 

A positive definite function $\kappa : \mathcal{X} \times \mathcal{X} \to \mathbb{R}$ define on a non-empty set $\mathcal{X}$ is called a kernel. A kernel function generalizes the notion of similarity between samples by mapping the points from the original feature space $\mathcal{X}$ into a different (usually richer, higher dimensional) Hilbert space $\mathcal{H}$ through a feature map $\phi : \mathcal{X} \to \mathcal{H}$ and then taking their inner product in $\mathcal{H}$. It holds that $\kappa(x, x') = \langle \phi(x), \phi(x') \rangle_\mathcal{H}$. 

The Gram matrix $K = [\kappa(x^{(i)}, x^{(j)})]_{i,j=1}^M$ is the matrix of pairwise kernel similarities. Given a p.d. kernel $\kappa$ and two sets $X_1 \in \mathbb{R}^{M\times P}, X_2 \in \mathbb{R}^{M'\times P}$ we can use the 'scipy' function 'cdist' to construct the $M \times M'$ Gram matrix. 

In [373]:
def build_kernel(kappa, X1, X2):
    return cdist(X1, X2, metric=kappa)

### Kernel eigendecomposition

We can analyze several properties of the kernel, such as the one arising from the integral operator
$$(T_\kappa)(f)(x) = \int_\mathcal{X} k(x, x') f(x') p(x') dx'$$
We can calculate the set of (possibly infinite) real, non-negative eigenvalues $\{ \eta_i \}$ and orthogonal eigenfunctions $\{ \phi_i \}$, leading to (Mercer theorem) an eigendecomposition of $\kappa$:
$$\kappa(x, x') = \sum_{i=1}^\infty \eta_i \phi_i(x) \phi_i(x').$$

The eigenvalue distribution can be approximated by one of the Gram matrix $k$ as $M \to +\infty$. The Hermitianity of the matrix $K$ guarantees that $K = \Phi \Lambda \Phi^\dagger$ where $\Phi = [\phi(x^{(i)})]_{i=1}^M$ is an unitary matrix and $\Lambda = \text{diag}(\lambda_0, ..., \lambda_{M-1})$ is a diagonal of eigenvalues sorted in descending order. We can use the following function to apply the decomposition:

In [374]:
def decompose_kernel(K, eigenvalue_descending_order=True, eigenvalue_removal_threshold=1e-12):
    """
    Decompose the kernel matrix K in its eigenvalues Λ and eigenvectors Φ
    :param K: kernel matrix, real and symmetric
    :param eigenvalue_descending_order: if True, the biggest eigenvalue is the first one
    :return: Lambda vector (n elements) and Phi matrix (N*N matrix)
    """
    Lambda, Phi = eigh(K)

    # set the desired order for the eigenvalues
    if eigenvalue_descending_order:
        Lambda = Lambda[::-1]
        Phi = Phi[:, ::-1]

    # kernel matrix is positive definite, any (small) negative eigenvalue is effectively a numerical error
    Lambda[Lambda < 0] = 0

    # remove smallest positive eigenvalues, as they are useless
    Lambda[Lambda < eigenvalue_removal_threshold] = 0

    return Lambda, Phi

### Kernel Ridge Regressor

We use the Kernel Ridge Regressor which is the function:
$$f^*(x) = \arg\min_{f \in H} \sum_{i=1}^M (f(x^{(i)}) - y^{(i)})^2 + \lambda \lVert f \rVert$$
whose function can be expressed as
$$f^*(x) = \sum_{i=1}^M \alpha^{(i)} \kappa(x, x^{(i)})$$
and 
$$\alpha = (K + \lambda I)^{-1} y.$$


In [399]:
def build_alpha_krr(kappa, the_X, the_y, lambda_coef):
    the_K = build_kernel(kappa, the_X, the_X)
    # solve with Least Square (needed if we incur in a singular matrix)
    the_alpha = scipy.linalg.lstsq(the_K + lambda_coef * np.eye(the_K.shape[0]), y.ravel())[0]
    # more effient solve
    # the_alpha = scipy.linalg.solve(the_K + lambda_coef * np.eye(the_K.shape[0]), y)
    return the_alpha

def build_krr(kappa, the_X, the_y, lambda_coef):
    the_alpha = build_alpha_krr(kappa, the_X, the_y, lambda_coef)
    def krr(the_X_test):
        K_test = build_kernel(kappa, the_X_test, the_X)
        return K_test @ the_alpha
    return krr

We can double check the correctness of the implementation by comparing it with sklearn KRR's one. 

In [400]:
from sklearn.kernel_ridge import KernelRidge
sklearn_krr = KernelRidge(kernel=np.inner, alpha=0.001)
sklearn_krr.fit(X, y)

# checking alpha coefficient matches
alpha_krr = sklearn_krr.dual_coef_
alpha_krr_sk = build_alpha_krr(np.inner, X, y, 0.001)
print(f"Avg difference in alpha coefficients: {np.linalg.norm(alpha_krr - alpha_krr_sk) / len(alpha_krr):0.8f}")

krr = build_krr(np.inner, X, y, 0.001)
y_pred = krr(X_dataset[20:25, :])
y_pred_sk = sklearn_krr.predict(X_dataset[20:25, :])
print(f"Avg difference in y predicted: {np.linalg.norm(y_pred - y_pred_sk) / len(y_pred_sk):0.8f}")

Avg difference in alpha coefficients: 0.00004599
Avg difference in y predicted: 0.00000067


### Kernel compatibility

#### Kernel-Target alignment

The kernel aligment has been proposed in [1] as a similarity measure between kernels:
$$A(K_1, K_2) = \frac{\langle K_1, K_2 \rangle}{\sqrt{\langle K_1, K_1 \rangle \langle K_2, K_2 \rangle}} \in [-1, +1]$$
We can use that to compare the kernel Gram matrix $K$ with the 'idealized' kernel made of the outer product of $y$ with itself:
$$A(K, y) = \frac{\langle K, yy^T \rangle}{\sqrt{\langle K, K \rangle \langle yy^T, yy^T \rangle}} = \frac{\langle K, yy^T \rangle}{M \sqrt{\langle K, K \rangle}}.$$

The Kernel-Target alignment give us an approximate indicator of the quality of our kernel on our specific problem. the uncentered kernel
alignment of Cristianini et al. (2001) does not correlate well with performance and thus, in general,
cannot be used effectively for learning kernels.

In [495]:
def kernel_target_alignment(K, y):
    Y = np.outer(y, y)
    norm = np.sqrt(np.sum(K * K)) * y.shape[0]
    return np.sum(K * Y) / norm

#### Centered Kernel-Target alignment


The Centered Kernel-Target alignment has been proposed in [2], and is computed as in (https://digitalcommons.uri.edu/cgi/viewcontent.cgi?article=2106&context=oa_diss).

Centering a kernel is exremely important

$$\mathrm{center}(K) = U K U^T, U = \Big[I - \frac{11^T}{M}\Big]$$

$$C(K, y) = A(\mathrm{center}(K), yy^T)$$

Can be used to guide an optimization process better. 

In [497]:
def center_kernel(K):
    m = Kc.shape[0]
    U = np.eye(m) - (1/m) * np.outer([1] * m, [1] * m)
    Kc = U @ K @ U.T
    return Kc

def centered_kernel_target_alignment(K, y):
    return kernel_target_alignment(center_kernel(K), y)

### Task-model alignment

We can rewrite each $g$ as a linear combination of the orthonormal functions $\{ \phi_\rho \}$, i.e.
$$g(x) = \sum_{\rho=0}^\infty a_\rho \phi_\rho(x), \qquad a_\rho = \int p(x) f(x) \phi_\rho(x) dx $$


By setting $\psi_\rho = \sqrt{\eta_\rho} \phi_\rho(x)$ we get an orthonormal basis. 

In the discrete case, the kernel is approximated by the matrix $K = [\kappa(x_i, x_j)]$. Due to the symmetric nature of $\kappa$, the matrix can be decomposed in $K = \Phi \Lambda \Phi^T$ where $\Lambda$ is a diagonal matrix of nonnegative real eigenvalues (in descending order) and $\Phi$ is an unitary matrix where each column is the corresponding eigenvector. We can set $\Psi = \Phi \Lambda^{1/2}$. 

We can then express the target function $\bar{f}$ and the predictor function $f^*$ in the form of $f(x) = \sum_\rho w_\rho \psi_\rho(x)$. In the finite setting, we recover the weight vector $w = \frac{1}{m} \Lambda^{-1/2} \Phi^T$.

In [480]:
def calculate_weight_coefficient(kernel_eigenvalues, kernel_eigenvectors, labels):
    """
    Calculates the weights of a predictor given the labels and the kernel eigendecomposition,
    as shown in (Canatar et al 2021, inline formula below equation 18).
    :param kernel_eigenvalues: vectors of m nonnegative eigenvalues 'eta'
    :param kernel_eigenvectors: vectors of m nonnegative eigenvectors 'phi'
    :param labels: vector of m labels corresponding to 'm' ground truth labels or predictor outputs
    :return: vector of m weights
    """
    # get the number of training elements
    m = kernel_eigenvalues.shape[0]

    # invert nonzero eigenvalues
    inv_eigenvalues = np.reciprocal(kernel_eigenvalues, where=kernel_eigenvalues > 0)

    # weight vectors are calculated by inverting formula: y = \sum_k=1^M w_k \sqrt{eta_k} \phi_k(x)
    the_w = (1 / m) * np.diag(inv_eigenvalues ** 0.5) @ kernel_eigenvectors.T @ labels
    the_w = (1 / m) * kernel_eigenvectors.T @ labels
    return the_w

def calculate_weight_coefficient_wo_eigenvalues(kernel_eigenvectors, labels):
    """
    Calculates the weights of a predictor given the labels and the kernel eigendecomposition,
    as shown in (Canatar et al 2021, inline formula below equation 18).
    :param kernel_eigenvalues: vectors of m nonnegative eigenvalues 'eta'
    :param kernel_eigenvectors: vectors of m nonnegative eigenvectors 'phi'
    :param labels: vector of m labels corresponding to 'm' ground truth labels or predictor outputs
    :return: vector of m weights
    """
    # get the number of training elements
    m = kernel_eigenvectors.shape[0]

    # weight vectors are calculated by inverting formula: y = \sum_k=1^M w_k \sqrt{eta_k} \phi_k(x)
    the_w = (1 / m) * kernel_eigenvectors.T @ labels
    return the_w

In [449]:
def cumulative_power_distribution(weight_coefficients, kernel_eigenvalues, rho):
    powers = kernel_eigenvalues * (weight_coefficients ** 2)
    return np.sum(powers[:rho]) / np.sum(powers)

In [490]:
X = [X[i] / np.linalg.norm(X[i]) for i in range(50)]
K = build_kernel(lambda x, z: np.exp(-0.01 * np.linalg.norm(x-z)), X, X)

In [502]:
Lambda, Phi = decompose_kernel(K)

In [503]:
Phi[1].dot(Phi[1])

1.0000000000000004

In [492]:
w = calculate_weight_coefficient(Lambda, Phi, y)
w0 = calculate_weight_coefficient_wo_eigenvalues(Phi, y)

In [493]:
w0 - w0

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [494]:
for i in range(10):
    print(f"{cumulative_power_distribution(w, Lambda, i): 0.6f}")

 0.000000
 0.827579
 0.832013
 0.841189
 0.852249
 0.860618
 0.860963
 0.871843
 0.873183
 0.873595


In [ ]:
# Big learning
https://arxiv.org/pdf/1302.4922.pdf
https://arxiv.org/pdf/2210.11836.pdf